In [ ]:
from set_math import *

In [ ]:
from set_visualization import *

# load

In [ ]:
% pwd

In [ ]:
df_test = pd.read_csv('train2.csv', index_col=0)

In [ ]:
df_test

In [ ]:
def make_year(date):
    return int(date.split('-')[0])

def make_month(date):
    return int(date.split('-')[1])

def make_day(date):
    return int(date.split('-')[2])

def make_monthday(date):
    return (date.split('-')[1:3])

In [ ]:
df_test['year'] = df_test['date'].apply(make_year)
df_test['month'] = df_test['date'].apply(make_month)
df_test['day'] = df_test['date'].apply(make_day)

In [ ]:
df_test

## 카테고리형 독립변수

In [ ]:
model_1 = sm.OLS.from_formula('units ~ C(item_nbr) + C(store_nbr) + C(dayofweek) + C(year) + C(month) + C(day)', data=df_test)
result_1 = model_1.fit()
print(result_1.summary())

# Leverage & Outlier

In [ ]:
# 1차
res = result_1.resid_pearson
index = result_1.resid.index
outlier_index = abs(res) >= 2

row = pd.Series(index=index, data=res)
drop_index = row[outlier_index].index

In [ ]:
df_test.drop(list(drop_index),inplace=True)

In [ ]:
model_1 = sm.OLS.from_formula('units ~ C(item_nbr) + C(store_nbr) + C(dayofweek) + C(year) + C(month) + C(day)', data=df_test)
result_1 = model_1.fit()
print(result_1.summary())

# 0.727 -> 0.804

In [ ]:
# 2차
res = result_1.resid_pearson
index = result_1.resid.index
outlier_index = abs(res) >= 2

row = pd.Series(index=index, data=res)
drop_index = row[outlier_index].index

In [ ]:
df_test.drop(list(drop_index),inplace=True)

In [ ]:
model_1 = sm.OLS.from_formula('units ~ C(item_nbr) + C(store_nbr) + C(dayofweek) + C(year) + C(month) + C(day) + 0', data=df_test)
result_1 = model_1.fit()
print(result_1.summary())

# 0.727 -> 0.804 -> 0.845

In [ ]:
# 3차
res = result_1.resid_pearson
index = result_1.resid.index
outlier_index = abs(res) >= 2

row = pd.Series(index=index, data=res)
drop_index = row[outlier_index].index

In [ ]:
df_test.drop(list(drop_index),inplace=True)

In [ ]:
model_1 = sm.OLS.from_formula('units ~ C(item_nbr) + C(store_nbr) + C(dayofweek) + C(year) + C(month) + C(day) + 0', data=df_test)
result_1 = model_1.fit()
print(result_1.summary())

# 0.727 -> 0.804 -> 0.845 -> 0.876

In [ ]:
# 4차
res = result_1.resid_pearson
index = result_1.resid.index
outlier_index = abs(res) >= 2

row = pd.Series(index=index, data=res)
drop_index = row[outlier_index].index

In [ ]:
df_test.drop(list(drop_index),inplace=True)

In [ ]:
model_1 = sm.OLS.from_formula('units ~ C(item_nbr) + C(store_nbr) + C(dayofweek) + C(year) + C(month) + C(day) + 0', data=df_test)
result_1 = model_1.fit()
print(result_1.summary())

# 0.727 -> 0.804 -> 0.845 -> 0.876 --> 0.901

In [ ]:
# 5차
res = result_1.resid_pearson
index = result_1.resid.index
outlier_index = abs(res) >= 2

row = pd.Series(index=index, data=res)
drop_index = row[outlier_index].index

In [ ]:
df_test.drop(list(drop_index),inplace=True)

In [ ]:
model_1 = sm.OLS.from_formula('units ~ C(item_nbr) + C(store_nbr) + C(dayofweek) + C(year) + C(month) + C(day) + 0', data=df_test)
result_1 = model_1.fit()
print(result_1.summary())

# 0.727 -> 0.804 -> 0.845 -> 0.876 -- 0.901 --> 0.919

In [ ]:
sp.stats.probplot(result_1.resid, plot=plt)
plt.show()

test = sms.omni_normtest(result_1.resid)
for xi in zip(['Chi^2', 'P-value'], test):
    print("%-12s: %6.3f" % xi)

# kfold

In [ ]:
df_test.columns

In [ ]:
X = pd.DataFrame(data=df_test, columns=['date', 'store_nbr', 'item_nbr', 'station_nbr', 'tmax', 'tmin',
       'tavg', 'depart', 'dewpoint', 'wetbulb', 'heat', 'cool', 'sunrise',
       'sunset', 'snowfall', 'preciptotal', 'stnpressure', 'sealevel',
       'resultspeed', 'resultdir', 'avgspeed', 'codesum', 'codesum_YN',
       'dayofweek', 'is_holiday'])
print(X.shape)
X.head()

In [ ]:
y = pd.DataFrame(data=df_test, columns=['units'])
print(y.shape)
y.head()

In [ ]:
from sklearn.model_selection import KFold

trial = 4

cv = KFold(n_splits=trial, shuffle=True, random_state=2)

total = 0


for train_index, test_index in cv.split(X):
    print("test index :", test_index, len(test_index))
    print("." * 80)
    print("train index:", train_index, len(train_index))
    print("=" * 80)

    X_train = X.loc[train_index, :]
    X_test = X.loc[test_index, :]
    y_train = y.loc[train_index, :]
    y_test = y.loc[test_index, :]

    model = sm.OLS.from_formula('units ~ resultspeed + cool + heat + preciptotal + depart + C(item_nbr) + C(store_nbr) + is_holiday', data=pd.concat([X_train, y_train], axis=1))
    result = model.fit()
    
    y_hat = result.predict(X_test)
    
    ESS = ((y_hat - y_hat.mean())**2).sum()
    TSS = ((y_test-y_test.mean())**2).sum()
    total += ESS/TSS
    print(ESS/TSS)
    
total/trial

# submission preparation

### test file preprocessing

In [ ]:
test = pd.read_csv('../../data/basic/test.csv')
print(test.shape)
test.head()

In [ ]:
key = pd.read_csv('../../data/basic/key.csv')
weather = pd.read_csv('../../data/weather_v8_with_holiday.csv')

In [ ]:
test = pd.merge(test, key)

In [ ]:
test = pd.merge(test, weather)

In [ ]:
test.columns

In [ ]:
test = pd.DataFrame(test, columns=['date','dayofweek', 'is_holiday', 'store_nbr', 'item_nbr', 'station_nbr', 'tmax',
       'tmin', 'tavg', 'depart', 'dewpoint', 'wetbulb', 'heat', 'cool',
       'sunrise', 'sunset', 'codesum', 'codesum_YN', 'snowfall', 'preciptotal',
       'stnpressure', 'sealevel', 'resultspeed', 'resultdir', 'avgspeed',
       ])

In [ ]:
test.head()

In [ ]:
def is_holiday(holiday):
    if holiday == 'holiday':
        return 1
    elif holiday == 'holiday_work':
        return 1
    elif holiday == "workday":
        return 0

In [ ]:
test['is_holiday'] = test['is_holiday'].apply(is_holiday)

In [ ]:
test['year'] = test['date'].apply(make_year)
test['month'] = test['date'].apply(make_month)
test['day'] = test['date'].apply(make_day)

In [ ]:
test.head()

In [ ]:
test_sample = test.drop(columns=['tmax', 'tmin', 'tavg', 'dewpoint', 'wetbulb',
       'sunrise', 'sunset','depart',  'codesum', 'codesum_YN', 'snowfall',
       'stnpressure', 'sealevel','resultdir', 'avgspeed',
       'station_nbr', 'heat', 'cool', 'preciptotal', 'is_holiday', 'resultspeed'])

In [ ]:
test_sample.head()

In [ ]:
test_X_df = pd.DataFrame(columns=['date', 'item_nbr', 'store_nbr','dayofweek', 'year', 'month', 'day'])

for nbr in range(1, 45+1):
    store_nbr = df_test[df_test['store_nbr'].isin([nbr])]
    sold_item_list = list(store_nbr['item_nbr'].unique())
    sold_item_list

    test_store_nbr = test_sample[test_sample['store_nbr'].isin([nbr])]
    to_test_df = test_store_nbr[test_store_nbr['item_nbr'].isin(sold_item_list)]

    test_X_df = pd.concat([test_X_df, to_test_df], axis = 0)
    
test_X_df.shape

In [ ]:
df_test.info()

In [ ]:
test_X_df.head()

In [ ]:
test_X_df.info()

In [ ]:
test_X_df['store_nbr'] = test_X_df['store_nbr'].astype(int)
test_X_df['item_nbr'] = test_X_df['item_nbr'].astype(int)
test_X_df['year'] = test_X_df['year'].astype(int)
test_X_df['month'] = test_X_df['month'].astype(int)
test_X_df['day'] = test_X_df['day'].astype(int)
test_X_df['dayofweek'] = test_X_df['dayofweek'].astype(object)

In [ ]:
test_result = result_1.predict(test_X_df)

# create submission file

In [ ]:
result = pd.concat([test, test_result], axis=1)

In [ ]:
print(result.shape)
result.head()

In [ ]:
result['units'] = result[0]

In [ ]:
def itiszero(units):
    if units > 0:
        return units
    else:
        return 0

In [ ]:
result['units'] = result['units'].apply(itiszero)

In [ ]:
result['units'] = result['units'].astype(int)

In [ ]:
result = pd.DataFrame(data=result, columns=['store_nbr', 'item_nbr', 'date', 'units'])

In [ ]:
for col in result[['store_nbr', 'item_nbr']]:
    result[col] = result[col].astype(int)

In [ ]:
result['id'] = result['store_nbr'].astype(str)+'_'+result['item_nbr'].astype(str)+'_'+result['date'].astype(str)

In [ ]:
for_submission = result.loc[:,['id','units']]

In [ ]:
for_submission.head()

In [ ]:
for_submission.to_csv('submission_sample4.csv', index=False)

# samplesubmission3
### dayofweek , outlier 5번 제거 ==> 캐글 0.26513
 


 
# samplesubmission4
### dayofweek, year, month, day , outlier 5번 제거 ==> 캐글 